In [ ]:
import boto3
import json
import os

sqs_client = boto3.client("sqs", endpoint_url="http://localhost:4566")
s3_client = boto3.client("s3", endpoint_url="http://localhost:4566")

In [ ]:
# CONFIGS
bucket_name = "dev-rcd-raw-video-files"
queue_name = "dev-terraform-queue-mdf-parser"

ROOT_FOLDER = "to_upload_files"
MESSAGES_FOLDER = "message_body"
S3_FILES_FOLDER = "s3_files"

s3_upload_folder = "datanauts"

url = sqs_client.get_queue_url(
    QueueName=queue_name,
)
queue_url = url["QueueUrl"]

sqs_messages_dir = os.path.join(ROOT_FOLDER, MESSAGES_FOLDER)
s3_files_dir = os.path.join(ROOT_FOLDER, S3_FILES_FOLDER)

In [ ]:
#SQS Message Uploader
sqs_file_names = os.listdir(sqs_messages_dir)

for file_name in sqs_file_names:
    message_path = os.path.join(sqs_messages_dir, file_name)
    message_file = open(message_path)
    message_body = str(json.load(message_file))

    #print(message_body)
    message = sqs_client.send_message(
        QueueUrl=queue_url,
        MessageBody=message_body
        )

In [ ]:
#S3 File Uploader
s3_file_names = os.listdir(s3_files_dir)

for file_name in s3_file_names:
    file_path = os.path.join(s3_files_dir, file_name)
    s3_path = os.path.join(s3_upload_folder,file_name)
    s3_client.upload_file(file_path, bucket_name, s3_path)

In [ ]:
for key in s3_client.list_objects(Bucket=bucket_name)['Contents']:
    print(key['Key'])
